# Experiment with Markov Chains

## Set up functions 

In [1]:
import numpy as np
from numpy.testing import assert_array_equal
from numpy.linalg import matrix_power, eig
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Some utils to verify the correctness of the array and matrix
def assert_initial_state(init_state):
    assert(isinstance(init_state, np.ndarray)), "`init_state` should be in numpy.ndarray type"
    assert(np.sum(init_state) == 1), "Sum of the vector `init_state` should be 1 but {} here.".format(np.sum(init_state))

def assert_transition_matrix(transition_matrix):
    assert(isinstance(transition_matrix, np.ndarray)), "`transition_matrix` should be in numpy.ndarray type"
    assert(len(transition_matrix.shape) == 2), "`transition_matrix` is only 2-D for this experiment"
    assert(transition_matrix.shape[0] == transition_matrix.shape[1]), "`transition_matrix` should be a squared matrix"
    assert_array_equal(x=np.sum(transition_matrix, 0), 
                       y=np.ones(transition_matrix.shape[0]),
                       err_msg="Columns in `transition_matrix` should sum up to 1",
                       verbose=True)

def get_state(init, transition, timestep):
    assert_initial_state(init)
    assert_transition_matrix(transition)
    assert(isinstance(timestep, int)), "Timestep must be an integer"
    return np.matmul(matrix_power(transition, timestep), init)

## Set initial state and check the evolution of the state probabilities

In [2]:
# Transition probability
prob_AB = 0.2
prob_BA = 0.6

# Stationary probability
prob_AA = 1 - prob_AB
prob_BB = 1 - prob_BA
transition_matrix = np.array([[prob_AA, prob_BA], 
                              [prob_AB, prob_BB]])
assert_transition_matrix(transition_matrix)
print("transition_matrix\n", transition_matrix)

transition_matrix
 [[0.8 0.6]
 [0.2 0.4]]


![](two-states.PNG)

In [3]:
init_state = np.transpose(np.array([0, 1]))
print('init_state', init_state)

init_state [0 1]


In [4]:
states = []
for timestep in range(0, 15):
    state = get_state(init_state, transition_matrix, timestep)
    states.append(state)
    
states_df = pd.DataFrame(states, columns=["state_A", "state_B"]) 
states_df

,state_A,state_B
0,0.000000,1.000000
1,0.600000,0.400000
2,0.720000,0.280000
3,0.744000,0.256000
4,0.748800,0.251200
5,0.749760,0.250240
6,0.749952,0.250048
7,0.749990,0.250010
8,0.749998,0.250002
9,0.750000,0.250000


## Verify the convergence solution

### Verify that the next distribution after the stationary is same as stationary

In [5]:
stationary = np.transpose(np.array([0.750000, 0.250000]))
result = np.matmul(transition_matrix, stationary)
print(result)

[0.75 0.25]


### Solution via eigenvector should be same as stationary distribution

In [6]:
first_eig_vector = eig(transition_matrix)[1][:,0]
normalised_eig = first_eig_vector/sum(first_eig_vector)  # Normalise to sum = 1
normalised_eig

array([0.75, 0.25])